## Loading Packages

In [17]:
from deeplearning import *
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from preprocessing import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from utils import *
import warnings

# plt.style.use('dark_background')
warnings.filterwarnings("ignore")
set_seed(42)

True

### Insuring that training is done on GPU if available

In [18]:
device = 'cpu'
if torch.cuda.device_count() > 0 and torch.cuda.is_available():
    print("Cuda installed! Running on GPU!")
    device = 'cuda'
else:
    print("No GPU available!")

No GPU available!


## Loading Data

In [7]:
path = "Data/"
universe_dict = universe_select(path, "Cu")

## Preprocessing

In [8]:
# Renaming the columns to price
universe_dict = price_rename(universe_dict)
# Cleaning the dataset of any erroneous datapoints
universe_dict = clean_dict_gen(universe_dict)
# Making sure that all the points in the window have consistent lenght
universe_dict = truncate_window_length(universe_dict)
df_full = generate_dataset(universe_dict, lg_returns_only=True)

Included Instrument:
cu_shfe
cu_lme
cu_comex_p
cu_comex_s
peso
sol
bdi
ted
vix
skew
gsci


The target column represents the log returns at one forecast length out in the future for the instrument of interest (aluminium or copper prices on the London Metals Exchange). 

To normalise the independent variables, the 1 day log returns between closing prices have been used.

In [9]:
# Visualise the plots
# visualise_universe(universe_dict)
df = df_full[["target"]]

In [10]:
df_full.head(5)
# df_full.tail(5)

,cu_shfe,cu_lme,cu_comex_p,cu_comex_s,peso,sol,bdi,ted,vix,skew,gsci,target
date,,,,,,,,,,,,
2006-09-13,0.001583,-0.013602,0.000590,0.055554,-0.000558,0.000307,0.029244,-0.004276,-0.064091,-0.048458,0.002624,-0.001404
2006-09-14,0.025695,-0.006098,-0.003100,-0.104881,0.000707,-0.001692,0.018715,0.005088,0.032559,-0.008086,-0.010694,0.021675
2006-09-15,-0.027710,-0.021468,-0.019557,0.041104,-0.001172,0.000154,0.012050,0.002636,0.018019,0.124060,-0.004952,0.043012
2006-09-18,0.006177,0.024153,0.030146,0.032495,0.000000,-0.000924,0.004920,0.002023,0.001699,-0.060186,0.007824,0.013450
2006-09-19,0.023214,0.007613,-0.012512,0.138083,0.000000,0.000000,-0.000935,-0.000202,0.016835,-0.018385,-0.019642,0.025225


Normalise data

Split into validation data and test data

Use validation to tune hyperparameters

Perform predictions on the test dataset

### Creating dataset for pytorch

In [52]:
df_X = df_full.loc[:, df_full.columns != 'target']
df_y = df_full.loc[:, df_full.columns == 'target']

In [122]:
class DeepLearning():
    """Class to perform training and validation for a given model"""
    def __init__(self, df_X, df_y,
                 #model, 
                 #optimiser, 
                 loss_function=nn.CrossEntropyLoss(),
                 device="cpu", seed=42):
        
        self.df_X = df_X
        self.df_y = df_y
   
        #self.model = model
        #self.optimiser = optimiser
        self.loss_function = loss_function
        self.device = device
        self.seed = seed
        
        assert (type(self.df_X) == pd.DataFrame)
        assert (type(self.df_y) == pd.DataFrame)
        assert (len(self.df_X.index) == len(self.df_y.index))
        assert (len(self.df_X.index) > 0)
        
        self.X_train = None
        self.X_val = None
        self.X_test = None
        
        self.y_train = None
        self.y_val = None
        self.y_test = None
        
        
    def train_val_test(self):
        """Splits the dataframes in to a training, validation
        and test set and creates torch tensors from the underlying
        numpy arrays"""
        # Splitting the sets into train, test and validation
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.df_X, self.df_y, test_size=0.2, shuffle=False)
        
        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(self.X_train, self.y_train, test_size=0.25, shuffle=False)

        print("Train Length: \t\t%i\nValidation Length: \t%i\nTest Length:\t\t%i" % (len(X_train), len(X_val), len(X_test)))
        
        # Tensor of training data
        self.X_train = torch.from_numpy(self.X_train.values).float()
        self.y_train = torch.from_numpy(self.y_train.values).float()

        # Tensor of training labels
        self.X_val = torch.from_numpy(self.X_val.values).float()
        self.y_val = torch.from_numpy(self.y_val.values).float()

        #  Tensor of test data
        self.X_test = torch.from_numpy(self.X_test.values).float()
        self.y_test = torch.from_numpy(self.y_test.values).float()

        # Size Check
        print("\nX Train Shape:\t\t", self.X_train.size())
        print("X Val Shape:\t\t", self.X_val.size())
        print("X Test Shape:\t\t", self.X_test.size())

        print("\ny Train Shape:\t\t", self.y_train.size())
        print("y Val Shape:\t\t", self.y_val.size())
        print("y Test Shape:\t\t", self.y_test.size())
        
    def normalise(self):
        """Normalizes the data using MaxMinScaler, which
            was chosen because it preserves the original 
            scale and doesn't reduce effect of outliers"""
        scaler = MinMaxScaler()
        
        # Normalize the validation and test set by the same
        # scale as the training data. Needed for training to
        # be correctly scaled
        
        self.X_train = scaler.fit_transform(self.X_train)
        self.X_val = scaler.transform(self.X_val)
        self.X_test = scaler.transform(self.X_test)
        
        """TODO implement output scaling for MTL because 
        otherwise the scale of the different outputs will
        lead to dominance of some tasks"""
        
    def train(self):
        return
    

In [123]:
# Hyperparameters
lr = 1e-2
momentum = 0.5
batch_size = 64
test_batch_size = 1000
n_epochs = 2
weight_decay=0

df_X = df_full.loc[:, df_full.columns != 'target']
df_y = df_full.loc[:, df_full.columns == 'target']

# optimiser = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

"""Do I need to declare a data explicitly in the dataset"""

learning = DeepLearning(df_X, df_y)
# Splitting the data into the train, validation and test sets
learning.train_val_test()
# Scaling the data by the training dataset
learning.normalise()

Train Length: 		1999
Validation Length: 	667
Test Length:		667

X Train Shape:		 torch.Size([1999, 11])
X Val Shape:		 torch.Size([667, 11])
X Test Shape:		 torch.Size([667, 11])

y Train Shape:		 torch.Size([1999, 1])
y Val Shape:		 torch.Size([667, 1])
y Test Shape:		 torch.Size([667, 1])


In [ ]:
class LSTMnet(nn.Module):
    """A Long Short Term Memory network
    model"""
    
    def __init__(self):
        super(LSTMnet, self, input_dim, hidden_dim, num_layers, dropout=0).__init__()
        
        # Input dimensions
        self.input_dim = input_dim
        
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.num_layers = num_layers
        
        # Dropout rate
        self.dropout = dropout

        # The output dimensions
        self.output_dim = output_dim
        
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers,
                            dropout=self.dropout, batch_first=True)

        # Output layer
        self.fc = nn.Linear(self.hidden_dim, self.output_dim)
        
        
    def forward(self, x)
    
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    
        def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

In [127]:
A = pd.DataFrame(data={"A": [1, 2, 5, 10, 15], "B": [0.1, 0.2, 0.3, 0.6, 0.9]})

# Results

In [659]:
df = df_full[["target"]]
# Taking t-1 to be the value for t
df["persistance"] = df.shift(1)
df.dropna(inplace=True)
# Calculating metrics for these columns
MSE, MAE, MDE = evaluate(df, "target", "persistance")

In [660]:
print(df[:5])
print(df[-5:])

              target  persistance
date                             
2006-09-14  0.021675    -0.001404
2006-09-15  0.043012     0.021675
2006-09-18  0.013450     0.043012
2006-09-19  0.025225     0.013450
2006-09-20  0.024843     0.025225
              target  persistance
date                             
2019-06-24 -0.000589     0.004104
2019-06-25 -0.026401    -0.000589
2019-06-26 -0.010480    -0.026401
2019-06-27 -0.011358    -0.010480
2019-06-28 -0.015457    -0.011358


In [661]:
# Placing in results dataframe
results = pd.DataFrame(columns={"MSE", "MAE", "MDE"})
results.index.name = 'Name'
results.head()
results.loc["persistance"] = [MSE, MAE, MDE] 
results.head()

,MSE,MAE,MDE
Name,,,
persistance,0.000582,0.017105,0.465585
